In [1]:
suppressMessages(library(ArchR))
library(here)
ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F)

addArchRThreads(threads = 15)

Warning message:
“package ‘gtable’ was built under R version 4.1.3”
Warning message:
“package ‘magrittr’ was built under R version 4.1.3”
Warning message:
“package ‘plyr’ was built under R version 4.1.3”
Warning message:
“package ‘stringr’ was built under R version 4.1.3”
Warning message:
“package ‘data.table’ was built under R version 4.1.3”
Warning message:
“package ‘Matrix’ was built under R version 4.1.3”
Warning message:
“package ‘Rcpp’ was built under R version 4.1.3”
Warning message:
“package ‘here’ was built under R version 4.1.3”
here() starts at /projects/pfenninggroup/singleCell/Pierce_transgen_rat_multiome


Attaching package: ‘here’


The following object is masked from ‘package:plyr’:

    here


Setting default number of Parallel threads to 15.



In [2]:
########################################
### load rheMac10 ArchR genome ###
GENOMEDIR='/home/bnphan/resources/genomes/rn7'
load(file.path(GENOMEDIR,'rn7_liftoff_mm10NcbiRefSeq_ArchR_annotations.rda'))

In [4]:
#################################################
## create Arrow file and compute duplicate scores
PROJDIR = 'Rat_Transgen_NAc_scATAC_Neuron'
proj = loadArchRProject(path = here('data/tidy_data/ArchRProjects',PROJDIR))

## drop the remaining INT and drop clusters
idxSample <- BiocGenerics::which(!proj$ClustersX201  %in%  c('C8', 'C13',  'C14', 'C15'))
cellsSample <- proj$cellNames[idxSample]

PROJDIR2 = here('data/tidy_data/ArchRProjects','Rat_Transgen_NAc_scATAC_MSN')
proj = subsetArchRProject(
  ArchRProj = proj,
  cells = cellsSample,
  outputDirectory = PROJDIR2,
  dropCells = TRUE,
  logFile = NULL,
  threads = getArchRThreads(),
  force = TRUE
)

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [5]:
for(varFeat in c(300)){
pd = getCellColData(proj)
dimRed = names(attributes(proj)$reducedDims)
embedNames = names(attributes(proj)$embeddings)

# # add iterative LSI
iterLSIName = paste0("IterativeLSIX",varFeat)

pdf()
proj <- addIterativeLSI( proj, useMatrix = "TileMatrix", 
name = iterLSIName,
LSIMethod = 2, 
iterations = 4, # increase this if noticing subtle batch effects
scaleTo = 15000, # median unique fragment per cell
selectionMethod = 'var',
clusterParams = list( # See Seurat::FindClusters
    resolution = c(.7, 1.3, 2, 3), # lower this if noticing subtle batch effects
    sampleCells = 10000,  n.start = 10), 
varFeatures = varFeat * 1000, # also can reduce this if noticing subtle batch effects
dimsToUse = 1:30, force = TRUE)
dev.off()
proj = saveArchRProject(ArchRProj = proj)

# add Harmony batch correction
HarmonyName = paste0("HarmonyX",varFeat)

print(HarmonyName)
proj <- addHarmony( proj, reducedDims = iterLSIName, 
                  max.iter.harmony = 20, name = HarmonyName, 
                  groupBy = c('Sample','Sire'), force = TRUE)

# add umap for harmony
UMAPName2 = paste0("UMAPX",varFeat)

print(UMAPName2)
proj <- addUMAP(proj, reducedDims = HarmonyName,  name = UMAPName2, 
                  nNeighbors = 30, minDist = 0.5, metric = "cosine", force = TRUE)

# add clusters for Harmony
ClustersName2 = paste0("ClustersX",varFeat)

print(ClustersName2)
proj <- addClusters( proj, reducedDims = HarmonyName, 
                     method = "Seurat", name = ClustersName2, 
                     filterBias = TRUE,resolution = 1, force = TRUE)

proj = saveArchRProject(proj)
}

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-232361a28c0a3-Date-2022-12-05_Time-07-33-00.log
If there is an issue, please report to github with logFile!

2022-12-05 07:33:01 : Computing Total Across All Features, 0.011 mins elapsed.

2022-12-05 07:33:06 : Computing Top Features, 0.085 mins elapsed.

###########
2022-12-05 07:33:06 : Running LSI (1 of 4) on Top Features, 0.098 mins elapsed.
###########

2022-12-05 07:33:06 : Sampling Cells (N = 10001) for Estimated LSI, 0.099 mins elapsed.

2022-12-05 07:33:06 : Creating Sampled Partial Matrix, 0.099 mins elapsed.

2022-12-05 07:34:33 : Computing Estimated LSI (projectAll = FALSE), 1.547 mins elapsed.

2022-12-05 07:43:29 : Identifying Clusters, 10.475 mins elapsed.

2022-12-05 07:43:48 : Identified 8 Clusters, 10.797 mins elapsed.

2022-12-05 07:43:48 : Saving LSI Iteration, 10.797 mins elapsed.

Warning message:
“package ‘uwot’ was built under R version 4.1.3”
2022-12-05 07:44:21 : Creating Cluster Matrix on 

[1] "HarmonyX300"


Harmony 1/20

Harmony 2/20

Harmony 3/20

Harmony converged after 3 iterations



[1] "UMAPX300"


08:29:47 UMAP embedding parameters a = 0.583 b = 1.334

08:29:47 Read 17409 rows and found 30 numeric columns

08:29:47 Using Annoy for neighbor search, n_neighbors = 30

08:29:47 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

08:29:49 Writing NN index file to temp file /tmp/RtmpeHMOCO/file232367e5632f1

08:29:49 Searching Annoy index using 32 threads, search_k = 3000

08:29:56 Annoy recall = 100%

08:29:56 Commencing smooth kNN distance calibration using 32 threads
 with target n_neighbors = 30

08:29:58 Initializing from normalized Laplacian + noise (using irlba)

08:29:58 Commencing optimization for 200 epochs, with 878866 positive edges

08:30:23 Optimization finished

08:30:23 Creating temp model dir /tmp/RtmpeHMOCO/dir23236242a3fa9

08:30:23 Creating dir /tmp/Rt

[1] "ClustersX300"


ArchR logging to : ArchRLogs/ArchR-addClusters-2323671547464-Date-2022-12-05_Time-08-30-25.log
If there is an issue, please report to github with logFile!

2022-12-05 08:30:27 : Running Seurats FindClusters (Stuart et al. Cell 2019), 0.002 mins elapsed.

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 17409
Number of edges: 684720

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.7145
Number of communities: 31
Elapsed time: 2 seconds


16 singletons identified. 15 final clusters.

2022-12-05 08:30:52 : Testing Biased Clusters, 0.413 mins elapsed.

2022-12-05 08:30:52 : Testing Outlier Clusters, 0.413 mins elapsed.

2022-12-05 08:30:52 : Assigning Outlier Clusters (n = 1, nOutlier < 5 cells) to Neighbors, 0.413 mins elapsed.

2022-12-05 08:30:52 : Assigning Cluster Names to 14 Clusters, 0.414 mins elapsed.

2022-12-05 08:30:52 : Finished addClusters, 0.415 mins elapsed.

Saving ArchRProject...

Loading ArchRProject...

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
           

In [6]:
proj = saveArchRProject(ArchRProj = proj)

Saving ArchRProject...

Loading ArchRProject...

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
         